In [1]:
import os
import pydicom
import numpy as np
import pyvista as pv
import vtk
from vtk.util import numpy_support
%gui qt

# Path to the folder containing DICOM slices
dicom_folder = r"UPENN-GBM\UPENN-GBM-00002\11-13-2001-NA-BRAINROUTINE-05064\11.000000-ep2dperf 3cc pre 12bolus-82091"

# List and sort DICOM files
dicom_files = sorted([f for f in os.listdir(dicom_folder) if f.endswith('.dcm') and f.startswith('02')])
print(f"Found {len(dicom_files)} DICOM files")

# Read DICOM slices and stack them into a 3D volume
dicom_data = [pydicom.dcmread(os.path.join(dicom_folder, f)) for f in dicom_files]
volume = np.stack([dcm.pixel_array for dcm in dicom_data], axis=-1).astype(np.float32)

# Normalize pixel values (scale between 0 and 1)
volume = (volume - volume.min()) / (volume.max() - volume.min())

# Print the volume shape to ensure it's 3D
print("Volume shape:", volume.shape)

# Create a vtkImageData object
imageData = vtk.vtkImageData()
nx, ny, nz = volume.shape  # Number of voxels along each axis
imageData.SetDimensions(nx, ny, nz)
imageData.SetSpacing(1, 1, 1)  # Change if you have real voxel spacing

# Convert the NumPy array to a VTK array (flattened in Fortran order)
vtk_array = numpy_support.numpy_to_vtk(
    volume.flatten(order='F'),
    deep=True,
    array_type=vtk.VTK_FLOAT
)
imageData.GetPointData().SetScalars(vtk_array)

# Wrap the vtkImageData in a PyVista object
grid = pv.wrap(imageData)

# Create the PyVista plotter and add an outline to visualize the volume bounds
p = pv.Plotter()
p.add_mesh(grid.outline(), color="black")
p.add_volume(grid, opacity="sigmoid", cmap="gray")

# Show the interactive window
p.show()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'UPENN-GBM\\UPENN-GBM-00002\\11-13-2001-NA-BRAINROUTINE-05064\\11.000000-ep2dperf 3cc pre 12bolus-82091'

Normalize dicom below using z score

In [3]:
import pydicom
import numpy as np
import os

# Function to normalize intensity values using Z-Score normalization
def z_score_normalization(dicom_file):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file)
    
    # Extract the pixel data (usually in the 'PixelData' attribute)
    pixel_data = dicom_data.pixel_array
    
    # Convert the pixel data to float for precision during normalization
    pixel_data = pixel_data.astype(np.float32)
    
    # Normalize using Z-Score (mean = 0, std = 1)
    mean_intensity = np.mean(pixel_data)
    std_intensity = np.std(pixel_data)
    
    normalized_pixel_data = (pixel_data - mean_intensity) / std_intensity
    
    # Convert back to the same dtype and encoding as the original data
    dicom_data.PixelData = normalized_pixel_data.astype(np.int16).tobytes()
    
    # Construct a valid file path for the normalized DICOM file
    normalized_dicom_file = os.path.join(
        os.path.dirname(dicom_file),
        f"normalized_{os.path.basename(dicom_file)}"
    )
    
    # Save the normalized DICOM file
    dicom_data.save_as(normalized_dicom_file)
    
    return dicom_data

# Function to normalize intensity values using Min-Max scaling
def min_max_normalization(dicom_file):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file)
    
    # Extract the pixel data (usually in the 'PixelData' attribute)
    pixel_data = dicom_data.pixel_array
    
    # Convert the pixel data to float for precision during normalization
    pixel_data = pixel_data.astype(np.float32)
    
    # Min-Max scaling (0 to 1 range)
    min_intensity = np.min(pixel_data)
    max_intensity = np.max(pixel_data)
    
    normalized_pixel_data = (pixel_data - min_intensity) / (max_intensity - min_intensity)
    
    # Convert back to the same dtype and encoding as the original data
    dicom_data.PixelData = (normalized_pixel_data * 255).astype(np.uint8).tobytes()
    
    # Construct a valid file path for the normalized DICOM file
    normalized_dicom_file = os.path.join(
        os.path.dirname(dicom_file),
        f"n{os.path.basename(dicom_file)}"
    )
    
    # Save the normalized DICOM file
    dicom_data.save_as(normalized_dicom_file)
    
    return dicom_data

# Example usage:
# Normalize using Z-Score normalization
normalized_dicom = z_score_normalization(r"D:\mlpr data\manifest-1669766397961\UPENN-GBM\UPENN-GBM-00001\02-06-2002-NA-BRAINROUTINE-33106\2.000000-t2Flairaxial ProcessedCaPTk-17693\1-35.dcm")

# Or normalize using Min-Max scaling
# normalized_dicom = min_max_normalization(r"D:\mlpr data\manifest-1669766397961\UPENN-GBM\UPENN-GBM-00001\02-06-2002-NA-BRAINROUTINE-33106\2.000000-t2Flairaxial ProcessedCaPTk-17693\1-01.dcm")


In [ ]:
import pydicom
import matplotlib.pyplot as plt

# Function to visualize DICOM image
def visualize_dicom(dicom_file):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file)
    
    # Extract pixel data
    pixel_data = dicom_data.pixel_array
    
    # Display the image using matplotlib
    plt.imshow(pixel_data, cmap=plt.cm.bone)  # 'bone' colormap is often used for medical images
    plt.axis('off')  # Turn off axis
    plt.show()

# Example usage:
dicom_file = r"D:\mlpr data\manifest-1669766397961\UPENN-GBM\UPENN-GBM-00001\02-06-2002-NA-BRAINROUTINE-33106\2.000000-t2Flairaxial ProcessedCaPTk-17693\n1-35.dcm"
visualize_dicom(dicom_file)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\mlpr data\\manifest-1669766397961\\UPENN-GBM\\UPENN-GBM-00001\\02-06-2002-NA-BRAINROUTINE-33106\\2.000000-t2Flairaxial ProcessedCaPTk-17693\\n1-35.dcm'

In [4]:
import os

# specify the folder path
folder_path = r'D:\mlpr data\radiomic_features_CaPTk'

# list all files in the folder
for filename in os.listdir(folder_path):
    # check if it's a file (not a directory)
    if os.path.isfile(os.path.join(folder_path, filename)):
        print(filename)


fe_params_UPENN-GBM_CaPTk.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_ap-rCBV_ED.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_ap-rCBV_ET.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_ap-rCBV_NC.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_PH_ED.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_PH_ET.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_PH_NC.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_PSR_ED.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_PSR_ET.csv
Radiomic_Features_CaPTk_automaticsegm_DSC_PSR_NC.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_AD_ED.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_AD_ET.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_AD_NC.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_FA_ED.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_FA_ET.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_FA_NC.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_RD_ED.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_RD_ET.csv
Radiomic_Features_CaPTk_automaticsegm_DTI_RD_NC.csv
Radiomic_Feature